In [1]:
import pandas as pd
import os

In [2]:
# list every file ending with .pdf in ./pdfs dir
os.listdir('./pdfs')
pdfs = [f for f in os.listdir('./pdfs') if f.endswith('.pdf')]
print(pdfs)


['billgraham-dorling-2dcir2006.pdf', 'dowjones-brdtradechi-sdny1982.pdf', 'peterman-v-republican-national-committee-dmont2019.pdf', 'fox-news-network-tveyes-02272018.pdf', 'campbell-acuff-1994.pdf', 'castlerock-carolpubl%E2%80%99g-2dcir1998.pdf', 'sonycorp-universal-1984.pdf', 'Golden%20v.%20Michael%20Grecco%20Prods%2019-cv-3156%20(E.D.N.Y.%20Mar.%209%202021).pdf', 'supermarket-sanfernando-9thcir1986.pdf', 'noland-janssen-sdny-2020.pdf', 'lennon-premisemedia-sdny2008.pdf', 'furie-infowars-cdcal2019.pdf', 'nat%E2%80%99lctr-riverside-cdcal2012.pdf', 'monsarrat-newman-1stcirc2022.pdf', 'rosemontenters-randomhouse-2dcir1966.pdf', 'solid-oak-sketches-2k-games-sdny2020.pdf', 'consumerunion-generalsignal-2dcir1983.pdf', 'nytimes-roxbury-dnj1977.pdf', 'brammer-violenthues-4thcir2019.pdf', 'rotbart-jrodwye-sdny1995.pdf', 'soc%E2%80%99yhtm-gregory-1stcir2012.pdf', "televisiondigest-ustelass'n-ddc1993.pdf", 'vhtinc-zillowgroup-9thcir2019.pdf', 'penguinrandomhouse-colting-sdny2017.pdf', 'cambridge

In [5]:
pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [172]:
# attempt to open pdfs using python
import PyPDF2
pdf_file = open('./pdfs/adjmi-dlt-sdny2015.pdf', 'rb')

In [181]:
# create a pdf reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [190]:
# print out text from the first page
pdf_reader.pages[0].extract_text().split('\n')

['Adjmi v. DLT Entm’t, Ltd., ',
 'No. 1:14-cv-00568-LAP (S.D.N.Y. Mar. 31, 2015) ',
 '    ',
 'Year 2015 ',
 'Court United States District Court for the Southern  District of New York  ',
 'Key ',
 'Facts Playwright David Adjmi authored 3C , a play that copies the plot premise, ',
 'characters, sets, and certain scenes from the popul ar 1970’s television series ',
 'Three’s Company .  DLT Entertainment Ltd. (“DLT”) owned the copyrig ht in ',
 'Three’s Company .  In 2012, DLT sent a “cease-and-desist” letter to  Adjmi ',
 'and the several production companies responsible fo r performances of 3C .  ',
 'The letter claimed 3C  infringed DLT’s copyright in Three’s Company  and ',
 'demanded Adjmi and others refrain from producing fu rther performances of ',
 'the play (among other demands).  In response, Adjmi  sought declaratory ',
 'judgment that 3C  does not infringe DLT’s copyright in Three’s Company .  In ',
 'support of his action, Adjmi claimed his copying as pects of Three’s Compan

### Setting up some functions for parsing all the PDFs to extract data into a JSON file

In [511]:
# check if line contains ONLY ' ' character
def contains_space(s):
    return all(c == ' ' for c in s)


# open pdf and get text content filtered into a dataframe row
def extract_text_from_pdf(pdf_filename):

    new_filename = pdf_filename.replace('.pdf','.txt')
    pdf_data = PyPDF2.PdfReader(open(pdf_filename, 'rb'))
    pdf_extracted = []
    for page in pdf_data.pages:
        pdf_extracted.extend(page.extract_text().split('\n'))
    # output pdf_extracted to a txt file
    with open('{}'.format(new_filename), 'w') as f:
        for line in pdf_extracted:
            if line.startswith('Year ') or line.startswith('Court ') or line.startswith('Holding ') or line.startswith('Key ') or line.startswith('Tags ') or line.startswith('Outcome') or line.startswith('Source: '):
                line = '\n' + line
            f.write(line + '\n')

    # I want to open the txt file and split along lines that are just whitespace
    with open(new_filename, 'r') as f:
        pdf_test = f.read().split('\n\n')
        
    pdf_test[0] = 'Title ' + pdf_test[0]
    # for each line in pdf_test, split the first word and the rest of the line
    pdf_values = [line.split(' ', 1) for line in pdf_test]
    # for line in pdf_values the first item is the column name and the second item is the value
    # I want to create a dictionary from this list
    pdf_dict = {k: v for k, v in pdf_values}
    # make a pandas df of it
    pdf_df = pd.DataFrame(pdf_dict, index=[0])
    pdf_df['Filename'] = pdf_filename.split('/')[-1]
    return pdf_df


## RUN FOR ALL PDF FILES!

In [512]:
import tqdm
counter = 0
for file in tqdm.tqdm(os.listdir('./pdfs/')):
    if file.endswith('.pdf'):
        new_df = extract_text_from_pdf('./pdfs/' + file)
        if counter == 0:
            pdf_main_df = new_df
            #print(pdf_main_df.columns)
        else:
            #print(new_df.columns)
            pdf_main_df = pd.concat([pdf_main_df, new_df], ignore_index=True)
        counter += 1

100%|██████████| 499/499 [00:04<00:00, 107.95it/s]


## We now have most of the data pulled

- need to do some cleanup but this will build a real database versus the index
- think about combining ETL steps from the other notebook here
- consider how having this in a SQL type database is different than the index

In [513]:
# Filling in blank years for text that wasn't extracted correctly
pdf_main_df.loc[pdf_main_df.Year.isna(), 'Year'] = pdf_main_df[pdf_main_df.Year.isna()]['Source:'].str.split('\n Year ').str[1].str.split('\n').str[0]
pdf_main_df.loc[pdf_main_df.Year.isna(), 'Year'] = pdf_main_df[pdf_main_df.Year.isna()]['Source:'].str.split(' Year ').str[1].str.split('\n').str[0]
pdf_main_df.loc[pdf_main_df.Year.isna(), 'Year'] = pdf_main_df[pdf_main_df.Year.isna()]['Title'].str.split(' Year ').str[1].str.split(' ').str[0]

assert(len(pdf_main_df[pdf_main_df.Year.isna()]) == 0)

# strip \n from Outcome column in each row
pdf_main_df['Outcome'] = pdf_main_df['Outcome'].str.strip(' \n')
pdf_main_df['Source:'] = pdf_main_df['Source:'].str.strip(' \n')
pdf_main_df['Tags'] = pdf_main_df['Tags'].str.strip(' \n')
pdf_main_df['Key'] = pdf_main_df['Key'].str.strip(' \n')
pdf_main_df['Holding'] = pdf_main_df['Holding'].str.strip(' \n')
pdf_main_df['Court'] = pdf_main_df['Court'].str.strip(' \n')
pdf_main_df['Year'] = pdf_main_df['Year'].str.strip(' \n')
pdf_main_df['Title'] = pdf_main_df['Title'].str.strip(' \n')

# lastly, rename some columns for clarity
pdf_main_df.rename(columns={'Source:': 'Source'}, inplace=True)
# Key to Key_Facts
pdf_main_df.rename(columns={'Key': 'KeyFacts'}, inplace=True)

In [514]:
pdf_main_df.KeyFacts.str.contains('Tags').sum()
pdf_main_df.Holding.str.contains('Tags').sum()
pdf_main_df.Court.str.contains('Tags').sum()
pdf_main_df.Year.str.contains('Tags').sum()
pdf_main_df.Title.str.contains('Tags').sum()
pdf_main_df.Source.str.contains('Tags').sum()
pdf_main_df.Outcome.str.contains('Tags').sum()

0

In [515]:
pdf_main_df.loc[pdf_main_df.Tags.isna(), 'Tags'] = pdf_main_df[pdf_main_df.Tags.isna()]['Holding'].str.split('\n Tags ').str[1].str.split(' ').str[0]
pdf_main_df.loc[pdf_main_df.Tags.isna(), 'Tags'] = pdf_main_df[pdf_main_df.Tags.isna()]['Title'].str.split(' Tags ').str[1].str.split(' ').str[0]


In [516]:
# There are some empty tag fields too...
len(pdf_main_df[pdf_main_df.Tags.isna()])

1

In [520]:
pdf_main_df[pdf_main_df.Title.isna()]


,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename


In [521]:
pdf_main_df[pdf_main_df.Year.isna()]


,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename


## gotta fix these

In [523]:
pdf_main_df[pdf_main_df.Court.isna()]


,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
88,"Lucasfilm Ltd. LLC, et al. v. Ren Ventures Ltd...",2018,NaN,NaN,NaN,Ninth,NaN,NaN,lucasfilm-renventures-9thcir2018.pdf
104,Estate of Anthony Barré and Angel Barré v. Car...,2017,NaN,NaN,NaN,Fifth,NaN,NaN,barre-carter-no.%2017-1057-072517.pdf
116,"Otto v. Hearst Communications, Inc. No. 1:17-c...",2018,NaN,NaN,NaN,Second,NaN,NaN,otto-hearst-communications-inc.no.1-17-cv-4712...
169,"Capitol Records, LLC v. ReDigi Inc. No. 16-232...",2018,NaN,NaN,NaN,Second,NaN,NaN,capitol-records-llc-redigi-inc-no.16-2321-2nd-...
229,"Authors Guild, Inc. v. Google Inc., No. 13-482...",2015,NaN,NaN,NaN,Second,NaN,NaN,authorsguild-google-2dcir2015.pdf


In [526]:
pdf_main_df[pdf_main_df.KeyFacts.isna()]

,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
88,"Lucasfilm Ltd. LLC, et al. v. Ren Ventures Ltd...",2018,NaN,NaN,NaN,Ninth,NaN,NaN,lucasfilm-renventures-9thcir2018.pdf
104,Estate of Anthony Barré and Angel Barré v. Car...,2017,NaN,NaN,NaN,Fifth,NaN,NaN,barre-carter-no.%2017-1057-072517.pdf
116,"Otto v. Hearst Communications, Inc. No. 1:17-c...",2018,NaN,NaN,NaN,Second,NaN,NaN,otto-hearst-communications-inc.no.1-17-cv-4712...
169,"Capitol Records, LLC v. ReDigi Inc. No. 16-232...",2018,NaN,NaN,NaN,Second,NaN,NaN,capitol-records-llc-redigi-inc-no.16-2321-2nd-...
229,"Authors Guild, Inc. v. Google Inc., No. 13-482...",2015,NaN,NaN,NaN,Second,NaN,NaN,authorsguild-google-2dcir2015.pdf


In [525]:
pdf_main_df[pdf_main_df.Holding.isna()]


,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
46,"Am. Geophysical Union v. Texaco, Inc., \n60 F...",1995,United States Court of Appeal s for the Second...,"Facts Plaintiffs, including A merican Geophys...",NaN,Second Circuit ; Education /Scholarship/Rese a...,Fair use not found,U.S. Copyright Office Fair Use Index. For mor...,am.geophysical-texaco-2dcir1995.pdf
88,"Lucasfilm Ltd. LLC, et al. v. Ren Ventures Ltd...",2018,NaN,NaN,NaN,Ninth,NaN,NaN,lucasfilm-renventures-9thcir2018.pdf
103,Ferdman v. CBS Interactive Inc. \nNo. 17 Civ ...,2018,United States District Court for the Southern ...,Facts Plaintiff Steven Ferdman is a photogra...,NaN,NaN,"Preliminary ruling, f air use not found , mixe...",U.S. Copyright Office Fair Use Index. For more...,ferdman-cbs-interactive-sdny2018.pdf
104,Estate of Anthony Barré and Angel Barré v. Car...,2017,NaN,NaN,NaN,Fifth,NaN,NaN,barre-carter-no.%2017-1057-072517.pdf
116,"Otto v. Hearst Communications, Inc. No. 1:17-c...",2018,NaN,NaN,NaN,Second,NaN,NaN,otto-hearst-communications-inc.no.1-17-cv-4712...
166,"TD Bank, N.A. v. Hill \nNo. 1.12-cv-07188-RBK-...",2015,United States District Court for the District ...,Facts Defendant Hill was a founder and former ...,NaN,Third Circuit; Textual work; Unpublished,Fair use not found,U.S. Copyright Office Fair Use Index. For mor...,tdbank-hill-dnj2015.pdf
169,"Capitol Records, LLC v. ReDigi Inc. No. 16-232...",2018,NaN,NaN,NaN,Second,NaN,NaN,capitol-records-llc-redigi-inc-no.16-2321-2nd-...
229,"Authors Guild, Inc. v. Google Inc., No. 13-482...",2015,NaN,NaN,NaN,Second,NaN,NaN,authorsguild-google-2dcir2015.pdf


In [524]:
## NOT SURE ABOUT THESE YET!
pdf_main_df[pdf_main_df.Tags.isna()]
# find if Tags is in any row in any column


,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
103,Ferdman v. CBS Interactive Inc. \nNo. 17 Civ ...,2018,United States District Court for the Southern ...,Facts Plaintiff Steven Ferdman is a photogra...,NaN,NaN,"Preliminary ruling, f air use not found , mixe...",U.S. Copyright Office Fair Use Index. For more...,ferdman-cbs-interactive-sdny2018.pdf


In [527]:
pdf_main_df[pdf_main_df.Outcome.isna()]

,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
25,"CCA and B , LLC v. F + W Media , Inc. , \n81...",2011,United States District Court for the Northern ...,Facts Plaintiff CCA and B published a book ...,The court held that the defendant’s us e of pl...,Eleventh Circuit; Parody/Satire ; Textual work...,NaN,U.S. Copyright Office Fair Use Index. For mor...,ccaandb-fwmedia-ndga2011.pdf
88,"Lucasfilm Ltd. LLC, et al. v. Ren Ventures Ltd...",2018,NaN,NaN,NaN,Ninth,NaN,NaN,lucasfilm-renventures-9thcir2018.pdf
104,Estate of Anthony Barré and Angel Barré v. Car...,2017,NaN,NaN,NaN,Fifth,NaN,NaN,barre-carter-no.%2017-1057-072517.pdf
116,"Otto v. Hearst Communications, Inc. No. 1:17-c...",2018,NaN,NaN,NaN,Second,NaN,NaN,otto-hearst-communications-inc.no.1-17-cv-4712...
141,"World Wrestling Fed’n Entm’t , Inc. v. Bozell...",2001,United States District Court for the Southern ...,Facts Plaintiff World Wrestling Federation En...,The court rejected defendants’ claim that they...,Second Circuit; Film/ Audiovisual ; Review/ Co...,NaN,U.S. Copyright Office Fair Use Index. For mor...,worldwrestling-bozell-sdny2001.pdf
147,"New Era Publ’ns Int’l , ApS v. Carol Publ’g G...",1990,United States Court of Appeals for the Second ...,Facts Plaintiff New Era Publications Interna...,The court found that defendant’s use of Hubba...,Second Circuit ; Education/Scholarship/Researc...,NaN,U.S. Copyright Office Fair Use Index. For mor...,newera-carolpubl%E2%80%99g-2dcir1990.pdf
150,"A.V. ex rel. Vanderhye v. iParadigms, L .L.C.,...",2009,United States Court of Appeals for the Fourth ...,"Facts Defendant iParadigms, LLC offered su...",The appeals court upheld th e district court :...,Fourth Circuit; Computer program; Education/S...,NaN,U.S. Copyright Office Fair Use Index. For mor...,a.v.-vanderhye-iparadigms-4thcir2009.pdf
169,"Capitol Records, LLC v. ReDigi Inc. No. 16-232...",2018,NaN,NaN,NaN,Second,NaN,NaN,capitol-records-llc-redigi-inc-no.16-2321-2nd-...
229,"Authors Guild, Inc. v. Google Inc., No. 13-482...",2015,NaN,NaN,NaN,Second,NaN,NaN,authorsguild-google-2dcir2015.pdf


In [528]:
pdf_main_df[pdf_main_df.Source.isna()]

,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename
88,"Lucasfilm Ltd. LLC, et al. v. Ren Ventures Ltd...",2018,NaN,NaN,NaN,Ninth,NaN,NaN,lucasfilm-renventures-9thcir2018.pdf
104,Estate of Anthony Barré and Angel Barré v. Car...,2017,NaN,NaN,NaN,Fifth,NaN,NaN,barre-carter-no.%2017-1057-072517.pdf
105,Camb ridge University Press v. Albert \nNo. 1...,2018,United States Court of Appeals for the Elevent...,Facts Plaintiff s are three academic publishi...,The Eleventh Circuit vacated the district cour...,,"Preliminary ruling, remand \n Source: U.S. C...",NaN,cambridge-university-pres-albert-11thcir2018.pdf
116,"Otto v. Hearst Communications, Inc. No. 1:17-c...",2018,NaN,NaN,NaN,Second,NaN,NaN,otto-hearst-communications-inc.no.1-17-cv-4712...
169,"Capitol Records, LLC v. ReDigi Inc. No. 16-232...",2018,NaN,NaN,NaN,Second,NaN,NaN,capitol-records-llc-redigi-inc-no.16-2321-2nd-...
175,"Sony Computer Entm’ t Am., Inc. v. Bleem, L .L...",2000,United States Court of Appeals for the Ninth C...,Facts Plaintiff Sony Computer Entertainment A...,"Vacating the lower court’s injunction, the cir...",Ninth Circuit ; Computer program,"Preliminary ruling, mixed result, or remand \...",NaN,sonycomputer-bleem-9thcir2000.pdf
197,"Righthaven , L.L.C. v. Jama, \nNo. 2:10- CV-1...",2011,United States District Court for the District ...,Facts Defendants were Center for Intercultur...,The court held that CIO’s use of the article w...,Ninth Circuit; Education/Scholarship/Research;...,Fair use found \n \nSource : U.S. Copyright O...,NaN,righthaven-jama-dnev2011.pdf
229,"Authors Guild, Inc. v. Google Inc., No. 13-482...",2015,NaN,NaN,NaN,Second,NaN,NaN,authorsguild-google-2dcir2015.pdf
245,Camb ridge University Press v. Becker \nNo. 1...,2020,United States District Court for the Northern ...,Facts Plaintiff s are three academic publishi...,"On remand, the court separately analyzed wh...",Education/Scholarship/Research; Internet/Digit...,Mixed result \n Source: U.S. Copyright Offic...,NaN,cambridge-university-press-becker-ndga2020.pdf


In [529]:
pdf_main_df[pdf_main_df.Filename.isna()]

,Title,Year,Court,KeyFacts,Holding,Tags,Outcome,Source,Filename


In [428]:
pdf_main_df.to_json('pdfs_extracted_03052025.json', index=False)
pdf_main_df.to_excel('pdfs_extracted_03052025.xlsx', index=False)